In [6]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader


import sentencepiece as spm
#import my_GPT
from mha_GPT import Mha_GPT
#import pickle
from torch.nn.utils.rnn import pad_sequence
import os
from dataset_bes import TinyDataset
#from TinyDataset import collate_fn
 
import datasets
from datasets import load_dataset
import wandb
# install for optimiser
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()
# Load the SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load("tinystories_tokeniser.model")

vocab = [sp.id_to_piece(i) for i in range(sp.get_piece_size())]


In [7]:
data = load_dataset("roneneldan/TinyStories")

train_data = data["train"]
val_data = data["validation"]
ds = dataset_bes.TinyDataset()


/home/pulsar/miniconda3/envs/sut/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
/home/pulsar/miniconda3/envs/sut/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [8]:
batch_size = 4
dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, collate_fn=ds.collate_fn)

In [21]:
# Hyperparameters
vocab_size = len(vocab)
embedding_size = 512
max_seq_length = 1194
num_heads = 4
#batch_size = 64  # You can adjust this based on your specific requirements

In [10]:
gpt_model = Mha_GPT(vocab_size, embedding_size, max_seq_length, num_heads)
criterion = torch.nn.CrossEntropyLoss()


In [11]:
from torch.optim import Adam
beta1 = 0.9
beta2 = 0.98
epsilon = 1e-9
learning_rate = 0.001  # Adjust this based on your experimentation

# Create Adam optimizer
optimizer = Adam(gpt_model.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=epsilon)


In [12]:
checkpoint_dir = 'checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)

In [13]:

wandb.login(key='0a9583c37867b372d3f0849f8385a8eb0d4c4ec2')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: birsenyildiz2018. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/pulsar/.netrc


True

In [22]:
wandb.init(project ='Birsen_GPT', entity = 'birsenyildiz2018',     
    config={
    "learning_rate": 0.001,
    "architecture": "Multi headed GPT",
    "dataset": "Tiny Stories",
    "epochs": 10, 
    })

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


acc_train,▁▄▆▇▆▇▇▇▇▇▇▇▇███████████████████████████
loss_train,▆█▃▅▆▅▃▇▄▃▅▄▅▄▃▄▄▄▄▁▄▄▄▅▄▄▁▃▃▃▅▃▄▄▄▅▃▄▃▆
perp_train,▃█▁▃▄▃▁▆▂▂▃▂▂▂▁▂▂▂▂▁▂▂▂▂▂▂▁▁▁▂▂▁▂▂▂▃▁▂▁▃
acc_train,0.48439
loss_train,3.84494
perp_train,46.7557


In [23]:
import math
import torch.nn.functional as F
milestones = [1000, 2000, 3000]

num_epochs = 10  # Adjust as needed
checkpoint_interval = 1
for epoch in range(num_epochs):
    correct_predictions = 0
    total_predictions = 0
    step = 1
    for batch in dl:
       
        tokens = batch['input']
        true_labels = batch['label']
        optimizer.zero_grad()
        output = gpt_model(batch['input'])
       
        if isinstance(output, tuple) and len(output) == 2:
            logits, attention_weights = output
        else:
        # Handle the case where your model returns only logits
             logits = output
             attention_weights = None  # You might need to adjust this based on your model's behavior
        
        model_output = logits.view(-1, vocab_size)  # Reshape to [batch_size * seq_length, num_classes]
        
        true_labels = true_labels.view(-1)  # Reshape to [batch_size * seq_length]
        loss = criterion(model_output, true_labels)
        max_indices = torch.argmax(model_output, dim=1)
        correct_predictions += ((max_indices - true_labels)==0).sum()
        total_predictions += len(true_labels)
        # correct_predictions += torch.sum(predictions==label)
        # total_predictions += batch_size
        acc = correct_predictions/total_predictions
        # acc = 0
        with autocast():
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        if step in milestones:
        # Save a checkpoint
           checkpoint_path = f'model_checkpoint_step_{step}.pth'
           torch.save(gpt_model.state_dict(), checkpoint_path)

        step += 1
    # Logging and checkpointing
        wandb.log({"acc_train": acc, "loss_train": loss.item(), "perp_train": math.exp(loss.item())})
        
        if (epoch + 1) % checkpoint_interval == 0:
             checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
             torch.save({
            'epoch': epoch,
            'model_state_dict': gpt_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_train': loss.item(),
            'acc_train': acc,
            'perp_train': math.exp(loss.item()),
            'batch_size': batch_size,
            'emd_dim': embedding_size,
            'num_heads': num_heads
          }, checkpoint_path)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Acc: {acc_train}, Train Loss: {loss.item()}")

RuntimeError: Parent directory checkpoints does not exist.

In [17]:
torch.save(gpt_model.state_dict(), 'mha_GPT.pth')

In [18]:
import torch.nn.functional as F
trained_gpt_model = Mha_GPT(vocab_size, embedding_size, max_seq_length, num_heads)
trained_gpt_model.load_state_dict(torch.load('mha_GPT.pth'))
trained_gpt_model.eval()


Mha_GPT(
  (embedding): Embedding(16000, 1024)
  (positional_encoding): PositionalEncoding()
  (multihead_self_attention): MultiheadSelfAttention(
    (linear_queries): ModuleList(
      (0-15): 16 x Linear(in_features=64, out_features=64, bias=True)
    )
    (linear_keys): ModuleList(
      (0-15): 16 x Linear(in_features=64, out_features=64, bias=True)
    )
    (linear_values): ModuleList(
      (0-15): 16 x Linear(in_features=64, out_features=64, bias=True)
    )
    (linear_out): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (add_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (feed_forward): FeedForward(
    (linear1): Linear(in_features=1024, out_features=10, bias=True)
    (relu): ReLU()
    (linear2): Linear(in_features=10, out_features=1024, bias=True)
  )
  (add_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc): Linear(in_features=1024, out_features=16000, bias=True)
)

In [24]:
token_ids = torch.tensor([sp.encode_as_ids("The apple")])
logits = gpt_model(token_ids)

In [25]:


print(logits.shape)
probs = F.softmax(logits, dim=-1)
print(probs)
probs = probs[0]
print(probs.shape)

torch.Size([1, 2, 16000])
tensor([[[1.8244e-06, 3.6753e-08, 7.4458e-04,  ..., 8.0007e-08,
          3.3161e-08, 2.6452e-08],
         [2.6648e-04, 2.5594e-11, 4.8363e-06,  ..., 1.1386e-10,
          1.9330e-11, 1.2572e-11]]], grad_fn=<SoftmaxBackward0>)
torch.Size([2, 16000])


In [26]:

print(vocab[1])
# Get the index of the word with the highest probability (argmax)
max_value, predicted_word_index = torch.max(probs[:, -1], dim=0)
print(predicted_word_index)
# You may want to convert this index back to the actual word using your vocabulary
predicted_word = vocab[predicted_word_index]
print(predicted_word)

<s>
tensor(0)
<unk>


In [19]:
def tell_me_a_story(text, counter):
    counter += 1
    token_ids = torch.tensor([sp.encode_as_ids(text)])
    
    output = trained_gpt_model(token_ids)
    
    if isinstance(output, tuple) and len(output) == 2:
        logits, attention_weights = output
    else:
        # Handle the case where your model returns only logits
        logits = output
        attention_weights = None  # You might need to adjust this based on your model's behavior

    
    probs = F.softmax(logits, dim=-1)
    predicted_word_index = torch.argmax(probs, dim=-1)
   
    predicted_word = sp.decode(predicted_word_index[0][-1].item())

    # print(predicted_word)
   
    if  counter == 100:
        return text
    else:
        return tell_me_a_story(text + " " + predicted_word, counter)

In [46]:
tell_me_a_story("In 1983 ", 0)

'In 1983  ex                                                                                                  '

In [20]:
tell_me_a_story("The red horse ", 0)

'The red horse  was very excited to play with her . She was very excited . She was very excited to go on her parents . She was so excited to go to go to go to go to go to go to her . She packed her mommy and the kitchen . She was so excited ! He was so excited ! He was so excited ! She was so excited ! He was so excited ! He was so excited ! He was so excited ! He was so excited ! He was so excited ! He was so excited'

In [27]:
tell_me_a_story("Red ", 0)

'Red  was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_attention_heatmap(attention_weights):
    sns.heatmap(attention_weights, cmap="viridis", annot=True, fmt=".2f", cbar=False)
    plt.show()

# Assuming attention_weights is a 2D NumPy array
plot_attention_heatmap(attention_weights.numpy())